In [2]:
!pip install apyori

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import pandas as pd
from apyori import apriori

In [31]:
base = pd.read_csv('./data/input.csv',delimiter=';' ,encoding='cp1252');
base

,Leite,CafÃ©,Cerveja,PÃ£o,Manteiga,Arroz,FeijÃ£o
0,NÃ£o,Sim,NÃ£o,Sim,Sim,NÃ£o,NÃ£o
1,Sim,NÃ£o,Sim,Sim,Sim,NÃ£o,NÃ£o
2,NÃ£o,Sim,NÃ£o,Sim,Sim,NÃ£o,NÃ£o
3,Sim,Sim,NÃ£o,Sim,Sim,NÃ£o,NÃ£o
4,NÃ£o,NÃ£o,Sim,NÃ£o,NÃ£o,NÃ£o,NÃ£o
5,NÃ£o,NÃ£o,NÃ£o,NÃ£o,Sim,NÃ£o,NÃ£o
6,NÃ£o,NÃ£o,NÃ£o,Sim,NÃ£o,NÃ£o,NÃ£o
7,NÃ£o,NÃ£o,NÃ£o,NÃ£o,NÃ£o,NÃ£o,Sim
8,NÃ£o,NÃ£o,NÃ£o,NÃ£o,NÃ£o,Sim,Sim
9,NÃ£o,NÃ£o,NÃ£o,NÃ£o,NÃ£o,Sim,NÃ£o


In [34]:
base = base.where(base == 'Sim')
for coluna in base.columns:
    base[coluna] = base[coluna].apply(lambda x: coluna if x == 'Sim' else x)

base

,Leite,CafÃ©,Cerveja,PÃ£o,Manteiga,Arroz,FeijÃ£o
0,NaN,CafÃ©,NaN,PÃ£o,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,PÃ£o,Manteiga,NaN,NaN
2,NaN,CafÃ©,NaN,PÃ£o,Manteiga,NaN,NaN
3,Leite,CafÃ©,NaN,PÃ£o,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,PÃ£o,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,FeijÃ£o
8,NaN,NaN,NaN,NaN,NaN,Arroz,FeijÃ£o
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [35]:
transacoes = []

for i in range(base.shape[0]):
    transacao = []
    for coluna in base.columns:
        valor = base.at[i, coluna]
        if pd.notna(valor):
            if valor == 'Sim':
                transacao.append(coluna)
            else:
                transacao.append("Não " + coluna)
    transacoes.append(transacao)

transacoes

[['Não CafÃ©', 'Não PÃ£o', 'Não Manteiga'],
 ['Não Leite', 'Não Cerveja', 'Não PÃ£o', 'Não Manteiga'],
 ['Não CafÃ©', 'Não PÃ£o', 'Não Manteiga'],
 ['Não Leite', 'Não CafÃ©', 'Não PÃ£o', 'Não Manteiga'],
 ['Não Cerveja'],
 ['Não Manteiga'],
 ['Não PÃ£o'],
 ['Não FeijÃ£o'],
 ['Não Arroz', 'Não FeijÃ£o'],
 ['Não Arroz']]

In [36]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.7)
saida = list(regras)
for regra in saida:
  print(regra)

RelationRecord(items=frozenset({'Não Manteiga', 'Não CafÃ©'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Não CafÃ©'}), items_add=frozenset({'Não Manteiga'}), confidence=1.0, lift=2.0)])
RelationRecord(items=frozenset({'Não PÃ£o', 'Não CafÃ©'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Não CafÃ©'}), items_add=frozenset({'Não PÃ£o'}), confidence=1.0, lift=2.0)])
RelationRecord(items=frozenset({'Não PÃ£o', 'Não Manteiga'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Não Manteiga'}), items_add=frozenset({'Não PÃ£o'}), confidence=0.8, lift=1.6), OrderedStatistic(items_base=frozenset({'Não PÃ£o'}), items_add=frozenset({'Não Manteiga'}), confidence=0.8, lift=1.6)])
RelationRecord(items=frozenset({'Não PÃ£o', 'Não Manteiga', 'Não CafÃ©'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Não CafÃ©'}), items_add=frozenset({'Não PÃ£o', 'Não Manteiga'}), confidence=1.0, lift=2.5), 

In [37]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [38]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
4,[Não CafÃ©],"[Não PÃ£o, Não Manteiga]",0.3,1.00,2.5
7,"[Não PÃ£o, Não Manteiga]",[Não CafÃ©],0.3,0.75,2.5
0,[Não CafÃ©],[Não Manteiga],0.3,1.00,2.0
1,[Não CafÃ©],[Não PÃ£o],0.3,1.00,2.0
5,"[Não Manteiga, Não CafÃ©]",[Não PÃ£o],0.3,1.00,2.0
6,"[Não PÃ£o, Não CafÃ©]",[Não Manteiga],0.3,1.00,2.0
2,[Não Manteiga],[Não PÃ£o],0.4,0.80,1.6
3,[Não PÃ£o],[Não Manteiga],0.4,0.80,1.6


In [23]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [24]:
regras = apriori(transacoes, min_support = 0.6, min_confidence = 0.7)
saida = list(regras) 
print(len(saida))
print(saida)

3
[RelationRecord(items=frozenset({'Não'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não'}), confidence=0.9090909090909091, lift=1.0)]), RelationRecord(items=frozenset({'Sim'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Sim'}), confidence=0.9090909090909091, lift=1.0)]), RelationRecord(items=frozenset({'Sim', 'Não'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Sim', 'Não'}), confidence=0.9090909090909091, lift=1.0), OrderedStatistic(items_base=frozenset({'Não'}), items_add=frozenset({'Sim'}), confidence=1.0, lift=1.1), OrderedStatistic(items_base=frozenset({'Sim'}), items_add=frozenset({'Não'}), confidence=1.0, lift=1.1)])]


In [25]:
print(saida[0])

RelationRecord(items=frozenset({'Não'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não'}), confidence=0.9090909090909091, lift=1.0)])


In [26]:
print(saida[1])

RelationRecord(items=frozenset({'Sim'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Sim'}), confidence=0.9090909090909091, lift=1.0)])


In [27]:
print(saida[2])

RelationRecord(items=frozenset({'Sim', 'Não'}), support=0.9090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Sim', 'Não'}), confidence=0.9090909090909091, lift=1.0), OrderedStatistic(items_base=frozenset({'Não'}), items_add=frozenset({'Sim'}), confidence=1.0, lift=1.1), OrderedStatistic(items_base=frozenset({'Sim'}), items_add=frozenset({'Não'}), confidence=1.0, lift=1.1)])


**Para facilitar a visualização, veja como transformar isso em um dataframe** 

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [28]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [29]:
RegrasFinais

,Antecedente,Consequente,suporte,confianca,lift
0,[Não],[Sim],0.909091,1.0,1.1
1,[Sim],[Não],0.909091,1.0,1.1


**Podemos ordenar estas regras por uma métrica desejada**

In [52]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
0,[Banana],[Shampoo],0.636364,1.000000,1.222222
1,[Shampoo],[Banana],0.636364,0.777778,1.222222
